In [2]:
# Añadir el directorio 'src' al sys.path
import sys
import os

In [3]:
# Obtener la ruta absoluta de la carpeta 'src' para que sea accesible desde el notebook
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [4]:
# Importar la función de preprocesamiento
from preprocessing import clean_text_spacy

In [5]:
# Importar pandas para cargar y manipular el dataset
import pandas as pd

In [6]:
# Cargar el dataset
df = pd.read_csv("../data/TA1C_dataset_detection_train.csv")

In [7]:
# Verificar las primeras filas del dataset
print(df.head())

              Tweet ID  Tweet Date Media Name Media Origin  \
0  1302968016477589504  07-09-2020    El País      Uruguay   
1  1296805148950515713  21-08-2020    El País      Uruguay   
2  1303065732884967426  07-09-2020    El País      Uruguay   
3  1302462441520074757  06-09-2020    El País      Uruguay   
4  1301763104435589120  04-09-2020    El País      Uruguay   

                                         Teaser Text  Tag Value  
0  #SegundaDivisión  | La fortaleza del ataque: R...         No  
1  Jorge Lanata a los argentinos que se van a Uru...         No  
2  Raffo: “Los montevideanos deben estar alerta p...         No  
3  Ecos del universo: joven uruguayo desentraña (...  Clickbait  
4  Propuesta quinquenal de ANEP: aumento de 3,8% ...         No  


In [8]:
# Aplicar preprocesamiento al texto (limpiar el 'Teaser Text')
df['cleaned_text'] = df['Teaser Text'].apply(clean_text_spacy)

In [9]:
# Mostrar algunas filas con el texto original y el texto limpio
print("\nEjemplos de texto original vs texto limpio:")


Ejemplos de texto original vs texto limpio:


In [10]:
print(df[['Teaser Text', 'cleaned_text']].head())

                                         Teaser Text  \
0  #SegundaDivisión  | La fortaleza del ataque: R...   
1  Jorge Lanata a los argentinos que se van a Uru...   
2  Raffo: “Los montevideanos deben estar alerta p...   
3  Ecos del universo: joven uruguayo desentraña (...   
4  Propuesta quinquenal de ANEP: aumento de 3,8% ...   

                                        cleaned_text  
0  segundadivisión    la fortaleza del ataque ram...  
1  jorge lanata los argentinos que se van uruguay...  
2  raffo " los montevideanos deben estar alerta p...  
3  ecos del universo joven uruguayo desentraña y ...  
4  propuesta quinquenal de anep aumento de   del ...  


In [11]:
# Guardar el dataset con el texto normalizado
df.to_csv("../data/TA1C_dataset_detection_train_cleaned.csv", index=False)
print("Dataset con texto limpio guardado en '../data/TA1C_dataset_detection_train_cleaned.csv'")

Dataset con texto limpio guardado en '../data/TA1C_dataset_detection_train_cleaned.csv'
